In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# --- 1. Cargar dataset ---
with open("dataset_mercadona.json", "r", encoding="utf-8") as f:
    data = json.load(f)

pares = data["pares"]
tickets = data["tickets"]

# --- 2. Crear vocabulario ---
productos = list({p for ticket in tickets for p in ticket})
productos.sort()
token2id = {p: i for i, p in enumerate(productos)}
id2token = {i: p for p, i in token2id.items()}
vocab_size = len(productos)

# --- 3. Dataset PyTorch ---
class ProductosDataset(Dataset):
    def __init__(self, pares, token2id):
        self.inputs = []
        self.targets = []
        for par in pares:
            entrada_ids = [token2id[p] for p in sorted(par["entrada"])]
            self.inputs.append(torch.tensor(entrada_ids, dtype=torch.long))
            self.targets.append(token2id[par["siguiente"]])
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

# --- 4. Collate function para padding ---
def collate_fn(batch):
    inputs, targets = zip(*batch)
    lengths = [len(seq) for seq in inputs]
    max_len = max(lengths)
    padded_inputs = torch.zeros(len(inputs), max_len, dtype=torch.long)
    for i, seq in enumerate(inputs):
        padded_inputs[i, :lengths[i]] = seq
    targets = torch.tensor(targets, dtype=torch.long)
    return padded_inputs, targets

dataset = ProductosDataset(pares, token2id)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [2]:
# --- 5. Definir transformer pequeño ---
class MiniTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=64, nhead=4, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # x: [batch, seq_len] -> transformer espera [seq_len, batch, d_model]
        x = self.embedding(x).permute(1,0,2)
        x = self.transformer(x)
        # predecimos solo el último token de la secuencia
        x = self.fc(x[-1])
        return F.log_softmax(x, dim=-1)

model = MiniTransformer(vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

/home/eponlop/apr/lib/python3.12/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [3]:
# --- 6. Entrenamiento ---
epochs = 50
for epoch in range(epochs):
    total_loss = 0
    for batch_inputs, batch_targets in dataloader:
        optimizer.zero_grad()
        output = model(batch_inputs)
        loss = criterion(output, batch_targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(dataloader):.4f}")

Epoch 1/50 - Loss: 2.5583
Epoch 2/50 - Loss: 2.2807
Epoch 3/50 - Loss: 2.2383
Epoch 4/50 - Loss: 2.2092
Epoch 5/50 - Loss: 2.1927
Epoch 6/50 - Loss: 2.1827
Epoch 7/50 - Loss: 2.1722
Epoch 8/50 - Loss: 2.1591
Epoch 9/50 - Loss: 2.1524
Epoch 10/50 - Loss: 2.1451
Epoch 11/50 - Loss: 2.1403
Epoch 12/50 - Loss: 2.1360
Epoch 13/50 - Loss: 2.1243
Epoch 14/50 - Loss: 2.1237
Epoch 15/50 - Loss: 2.1152
Epoch 16/50 - Loss: 2.1090
Epoch 17/50 - Loss: 2.1072
Epoch 18/50 - Loss: 2.1002
Epoch 19/50 - Loss: 2.0963
Epoch 20/50 - Loss: 2.0921
Epoch 21/50 - Loss: 2.0883
Epoch 22/50 - Loss: 2.0837
Epoch 23/50 - Loss: 2.0825
Epoch 24/50 - Loss: 2.0708
Epoch 25/50 - Loss: 2.0706
Epoch 26/50 - Loss: 2.0688
Epoch 27/50 - Loss: 2.0643
Epoch 28/50 - Loss: 2.0583
Epoch 29/50 - Loss: 2.0557
Epoch 30/50 - Loss: 2.0531
Epoch 31/50 - Loss: 2.0476
Epoch 32/50 - Loss: 2.0415
Epoch 33/50 - Loss: 2.0369
Epoch 34/50 - Loss: 2.0368
Epoch 35/50 - Loss: 2.0294
Epoch 36/50 - Loss: 2.0296
Epoch 37/50 - Loss: 2.0267
Epoch 38/5

In [14]:
# --- 7. Ejemplo de predicción ---
def predecir_siguiente(model, secuencia, token2id, id2token):
    model.eval()
    secuencia_ordenada = sorted(secuencia)  # ✅ ordenar entrada
    entrada_ids = torch.tensor([[token2id[p] for p in secuencia_ordenada]], dtype=torch.long)
    with torch.no_grad():
        output = model(entrada_ids)
        probs = output.exp().squeeze()
        # ✅ filtrar productos ya comprados
        for p in secuencia_ordenada:
            probs[token2id[p]] = 0.0
        probs = probs / probs.sum()
        idx = torch.argmax(probs).item()
    return id2token[idx]


ejemplo = ["Arroz redondo", "Sal fina", "Pechuga de pollo", "Chocolate negro", "Plátanos"]
print("Entrada:", ejemplo)
print("Predicción siguiente producto:", predecir_siguiente(model, ejemplo, token2id, id2token))

Entrada: ['Arroz redondo', 'Sal fina', 'Pechuga de pollo', 'Chocolate negro', 'Plátanos']
Predicción siguiente producto: Yogur natural
